# Likelihood-based models

This seminar will be about likelihood-based models: autoregressive and flow-based. Seminar consists of 3 parts:
- Likelihood model in 1D - fitting histogram using SGD
- Deep Autoregressive model in 2D
- Detaled study of Real NVP model in 2D 

# Part 1. 1D warmup

In this part we will build our first likelihood-based model for 1D data and will try to fit it using gradient methods.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torch.utils.data
import math
from sklearn.model_selection import train_test_split
import random

Choose your device: don't forget to switch to GPU runtime when working in collab with cuda.

In [ ]:
device = 'cuda'

First, we define the procedure of data generation. It will generate a dataset of samples $x \in \{0 \dots 99\}$

In [ ]:
def sample_data():
    count = 10000
    rand = np.random.RandomState(0)
    a = 0.3 + 0.1 * rand.randn(count)
    b = 0.8 + 0.05 * rand.randn(count)
    mask = rand.rand(count) < 0.5
    samples = np.clip(a * mask + b * (1 - mask), 0.0, 1.0)
    
    return np.digitize(samples, np.linspace(0.0, 1.0, 100))

We generate data and perform train/val/test split.

In [ ]:
data = sample_data()
train_data, test_data = train_test_split(data, test_size = 0.3)
train_data, val_data = train_test_split(train_data, test_size = 0.3)

Let's plot and visualize the histogram of training data!

In [ ]:
def plot_histogram(data):
    counts = Counter(data)
    keys = list(counts.keys())
    values = list(counts.values())
    plt.bar(keys, values)
    plt.show()

In [ ]:
plot_histogram(train_data)

On lecture we have discussed how to build histogram model. But this model is not the best choice for high-dimensional data. So, we suggesst to you to implement the following parametrized model:

$$ p_\theta(x)_i = \frac{e^{\theta_i}}{\sum_j{e^{\theta_j}}} $$

Where $\theta=(\theta_0 \dots \theta_{99})$

We propose you to implement this model in the following class

In [ ]:
class SimpleProbabilityModel(nn.Module):
    # Store all parameters of your model as class fields in constructor
    def __init__(self,  num_elements=100):
        super(SimpleProbabilityModel, self).__init__()
        
        ################
        # YOUR CODE HERE
        ###############
        
    # Forward should return vector of log probabilities for each element
    def forward(self):
        ################
        # YOUR CODE HERE
        ###############
    
    # Should sample element using probabilities, obtained from parameters. Return single number 0..99
    def sample(self):
        ################
        # YOUR CODE HERE
        ###############

We will train this model using negative log-likelihood optimization: $ L_i = -\log p_{y_i} $. Implement this loss calculation for your model given a batch of data samples.

In [ ]:
# data: n.array of numbers from your training distribution
# model: instance of your SimpleProbabilityModel.
# should return: negative log-likelihood of your data given the model to perform backpropagation
def calc_loss(data, model):
    ################
    # YOUR CODE HERE
    ###############

Finally, we can create instance of our model and perform training. Note that if your calculated previous loss as classic natural logarithm, here we scale it to binary logarithm for logging likelihood in bits (which is better for interpretation and comparisons).

In [ ]:
model = SimpleProbabilityModel().to(device)

In [ ]:
def train_simple_model(model, train_data, val_data, num_epochs=10000, batch_size=4000, lr=0.01):
    optimizer = optim.SGD(model.parameters(), lr=lr)
    train_losses = []
    val_losses = []
    for i in range(num_epochs):
        for j in range(len(train_data) // batch_size):
            optimizer.zero_grad()
            batch = train_data[batch_size * j:batch_size * (j + 1)]
            l = calc_loss(batch, model)
            train_losses.append(l.item() / math.log(2))
            l.backward()
            optimizer.step()
        l = calc_loss(val_data, model)
        val_losses.append(l.item() / math.log(2))
    
    print("Train NLL(bits)")
    plt.plot(train_losses, color='green')
    plt.show()

    print("Val NLL(bits)")
    plt.plot(val_losses, color='red')
    plt.show()
    
    print("Final validation NLL(bits): {}".format(val_losses[-1]))

In [ ]:
train_simple_model(model, train_data, val_data)

You can also tune your training parameters (number of epochs, batch size, learning rate, optimizer), to improve validation NLL. You should obtain something below 6.

Finally, let's sample values from our model and visualize histograms of our test data and our sample data.

In [ ]:
sampled_data = [model.sample() for _ in range(len(test_data))]

In [ ]:
plot_histogram(sampled_data)
plot_histogram(test_data)

Training here might not yield perfect results, but pictures should look at least similar.

# Part 2. 2D discrete data. Autoregressive model

In this part we will built our own autoregressive model to work with two-dimensional discrete data data. 

We will load 2D distribution as is from file. It's 200x200 numpy array with probabilities.

In [ ]:
original_distribution = np.load("distribution.npy")

Let's define class to sample pair of numbers $(x,y) \in \{0 \dots 199\}^2$ from this distribution.

In [ ]:
class SampleDist:
    def __init__(self, distribution):
        self.probabilities = distribution.flatten()
        self.rows, self.cols = distribution.shape
        self.values = np.array([[i // self.cols, i % self.cols] for i in range(self.rows * self.cols)])

    def sample(self):
        idx = np.random.choice(self.rows * self.cols, p = self.probabilities)
        
        return self.values[idx]

So, we define distribution, sample data and create train/val/test splits.

In [ ]:
dist2D = SampleDist(original_distribution)

In [ ]:
SIZE = 100000
sampled_data = np.array([dist2D.sample() for _ in range(SIZE)])

In [ ]:
train_data, test_data = train_test_split(sampled_data, test_size = 0.2)
train_data, val_data = train_test_split(train_data, test_size = 0.2)

We will build our autoregressive model in $(x, y)$ as follows:

- Train marginal model $p(x)$ as in part 1
- Create and train conditional model $p(y|x)$ as multi-layer neural network

Here, create class for your conditional model $p(y|x)$. It should take $x$ as batch of integer inputs and return batch of probability distributions over $y$.

In [ ]:
class ConditionalModel(nn.Module):
    # Store all your trainable layers as model fiels in constuctor
    def __init__(self):
        super(ConditionalModel, self).__init__()
        
        ################
        # YOUR CODE HERE
        ###############
    
    # Forward pass takes LongTensor x of shape (N,) and should return predicted logprobs of shape (N, 200)
    def forward(self, x):
        ################
        # YOUR CODE HERE
        ###############

Finally, create a model and train it.

In [ ]:
cond_model = ConditionalModel().to(device)

In [ ]:
def train_cond_model(cond_model, train_data, num_epochs=100, lr=0.001, batch_size=10000):
    dataset = torch.utils.data.TensorDataset(torch.LongTensor(train_data.T[0]).to(device), 
                                             torch.LongTensor(train_data.T[1]).to(device))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    loss = nn.NLLLoss()
    optimizer = optim.Adam(cond_model.parameters(), lr=lr)
    
    train_losses = []
    
    for i in range(num_epochs):
        for X_train, Y_train in dataloader:
            optimizer.zero_grad()
            predictions = cond_model(X_train)
            l = loss(predictions, Y_train)
            l.backward()
            optimizer.step()
            
            train_losses.append(l.item() / math.log(2))
    
    print("Train NLL(bits)")
    plt.plot(train_losses, color='green')
    plt.show()

In [ ]:
train_cond_model(cond_model, train_data)

To build compound model, we will also need our simple model from part 1, trained on marginal data from our distribution (only x values).

In [ ]:
x_model = SimpleProbabilityModel(num_elements=200).to(device)
train_simple_model(x_model, train_data.T[0], val_data.T[0], num_epochs=3000)

Finally, we are ready to build compound model for our total $(x, y)$ distribution modeling. Having two trained models implement sampling procedure and probability calculation.

In [ ]:
class CompoundModel:
    def __init__(self, x_model, cond_model):
        self.x_model = x_model
        self.cond_model = cond_model
        
        self.x_model.eval()
        self.cond_model.eval()
    
    # Given two numbers x, y from 0 .. 199, return NLL value -log p(x,y)
    # Normalize in in the way it will return NLL in bits / dimention (binary log divided by two in our case)
    def get_logprob(self, x, y):
        ################
        # YOUR CODE HERE
        ###############
    
    # Implement sampling procedure. One call should return sample (x, y) as numpy array from two elements
    def sample(self):
        ################
        # YOUR CODE HERE
        ###############

In [ ]:
compound_model = CompoundModel(x_model, cond_model)

Calculate total average NLL in bits / dimension on your validation data. Tune training parameters and conditional model architecture to boost your performance.

In [ ]:
total_logprob = 0
for elem in val_data:
    logprob = compound_model.get_logprob(elem[0], elem[1])
    total_logprob += logprob
print("Total NLL on validation data per dimension: {}".format(total_logprob / val_data.shape[0]))

Check if sampling from your model works.

In [ ]:
compound_model.sample()

Finally, get enough samples from your final model and display 2D histogram of the results. Compare them with the results you can get from your test data.

In [ ]:
sampled_2d_data = np.array([compound_model.sample() for _ in range(test_data.shape[0])])

In [ ]:
def plot_2dhistogram(data):
    plt.hist2d(data.T[0], data.T[1], bins=200, cmap='gray')
    plt.show()

In [ ]:
plot_2dhistogram(sampled_2d_data)
plot_2dhistogram(test_data)

Doesn't this picture resemble anything? (look at the rotated version of the histogram). Check out how your original distribution looks like!

In [ ]:
plt.imshow(original_distribution, cmap="gray")
plt.show()

# Part 3. 2D Continuous Data. RealNVP flow model

In this part we will build flow model, transforming given 2D continuous distribution to uniform 2D distribution. Our distribution will be represented as a set of samples. We will use RealNVP model, which we discussed on lecture.

First of all, we define function for data sampling, use it and perform train/val/test split as usual. This time all elements also have labels, associated with them. We wil use them for visualization purposes.

In [ ]:
def sample_data():
    count = 100000
    rand = np.random.RandomState(0)
    a = [[-1.5, 2.5]] + rand.randn(count // 3, 2) * 0.2
    b = [[1.5, 2.5]] + rand.randn(count // 3, 2) * 0.2
    c = np.c_[2 * np.cos(np.linspace(0, np.pi, count // 3)),
    -np.sin(np.linspace(0, np.pi, count // 3))]

    c += rand.randn(*c.shape) * 0.2
    data_x = np.concatenate([a, b, c], axis=0)
    data_y = np.array([0] * len(a) + [1] * len(b) + [2] * len(c))
    perm = rand.permutation(len(data_x))
    return data_x[perm], data_y[perm]

In [ ]:
X, Y = sample_data()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2)

Let's take a look on a 2D histogram of our train set distribution.

In [ ]:
plt.hist2d(X_train.T[0], X_train.T[1], bins=100, cmap='gray')
plt.show()

Let's also visualize distribution in point form with different colors, associated with labels.

In [ ]:
plt.scatter(X_train.T[0], X_train.T[1], c = Y_train)
plt.show()

We will build RealNVP flow model using affine coupling layer as main building block. In two-coordinates case it takes the following form:

$$ (x_1, x_2) \rightarrow (z_1, z_2) $$
$$ z_1 = x_1 $$
$$ z_2 = x_2 \cdot \sigma(s(x_1)) + t(x_1) $$

Where vector from $s, t$ values is calculated as neural network with $x_1$ on input and $\sigma$ is sigmoid function. Cool thing about RealNVP is that this network can be arbitrary complex and doesn't need to be invertible itself to have invertible flow for affine coupling layer!

Flows support composition and we will use it. In general, we will compose our total flow in the following way:

$$ (z_1, z_2) = (\sigma \circ f_{\theta, 1} \circ \dots \circ f_{\theta, n})(x_1, x_2) $$

Where $f_{\theta, i}$ is affine coupling layers described above. And $\sigma$ is elementwise sigmoid function

Important thing in flows training is loss function. In case of flows composition, log-likelihood objective takes the following form:

$$ \log p_\theta(x) = \log p_\theta(z) + \sum_i \log | \det \frac{\partial f_i}{\partial f_{i-1}}| $$

Here $\det \frac{\partial f_i}{\partial f_{i-1}}$ is Jacobian matrix determinant for each flow operation. And $\log p_\theta(z)$ is log likelihood of latent space we are mapping to (in our case $U(0, 1)^2$)

Answer the following questions (you will need the answers to complete your RealNVP model code):

1. What is $\log | \det \frac{\partial f_i}{\partial f_{i-1}} |$ for affine coupling operation?
2. What is $\log | \det \frac{\partial f_i}{\partial f_{i-1}} |$ for elementwise sigmoid operation?
3. What are the inverse operations for both sigmoid and affine coupling?
4. How $\log p_\theta(z)$ in case of $U(0, 1)^2$ can be calculated?

We will start implementation of RealNVP from affine coupling layer. You will need to implement inverse operation calculations as well as objective update for each layer ($\log | \det \frac{\partial f_i}{\partial f_{i-1}} |$).

In [ ]:
class AffineCoupling(nn.Module):
    def __init__(self, num_features=2):
        super(AffineCoupling, self).__init__()
        # Store your NN model for s and t values calculation
        # Your network should take num_features // 2 elements on input (1 in our case)
        # And return num_features elements on output (1 in our case)
        self.NN = # YOUR CODE HERE
    
    # Forward pass is implemented for you. Implement your objective update
    def forward(self, x, objective):
        z1, z2 = torch.chunk(x, 2, dim=1)
        h = self.NN(z1)
        shift = h[:, 0::2]
        
        scale = torch.sigmoid(h[:, 1::2])
        z2 += shift
        z2 *= scale
        objective += # YOUR CODE HERE
        
        return torch.cat([z1, z2], dim=1), objective
    
    # Implement reverse value calculation
    def reverse(self, x):
        ################
        # YOUR CODE HERE
        ###############

Now do the same for elemntwise sigmoid function.

In [ ]:
# Elementwise sigmoid flow from R^n to (0; 1)^n
class ElementwiseSigmoid(nn.Module):
    def __init__(self):
        super(ElementwiseSigmoid, self).__init__()
    
    def forward(self, x, objective):
        x = torch.sigmoid(x)
        objective += # YOUR CODE HERE
        return x, objective
    
    def reverse(self, x):
        ################
        # YOUR CODE HERE
        ###############

We will also need to swap dimentions from time to time for affine couling layer, so it leaves different parts of input the same on different iterations. Answer the following questions:
1. Is this operation legal? Is it a flow? What is the reverse operation?
2. That is $\log | \det \frac{\partial f_i}{\partial f_{i-1}} |$ for this operation?

In [ ]:
def dimensions_swap(x):
    x1, x2 = torch.chunk(x, 2, dim=1)
    return torch.cat([x2, x1], dim=1)

Finally, we build RealNVP model from defined operations. Feel free to change number of affine coupling layers inside the model.

In [ ]:
class RealNVP(nn.Module):
    def __init__(self, n_transforms=4):
        super(RealNVP, self).__init__()
        
        self.affines = nn.ModuleList([AffineCoupling() for _ in range(n_transforms)])
        self.sigmoid = ElementwiseSigmoid()
        
    def forward(self, x):
        objective = 0
        for layer in self.affines:
            x, objective = layer(x, objective)
            x = dimensions_swap(x)

        x, objective = self.sigmoid(x, objective)
        
        return x, objective
    
    def reverse(self, x):
        x = self.sigmoid.reverse(x)
        for layer in reversed(self.affines):
            x = dimensions_swap(x)
            x = layer.reverse(x)
            
        return x

At last, we define training procedure. Here we have calculated our total objective from flows log-det-s, however, we still need to define our loss, which we need to minimize. We also want to accumulate training loss values in bits. Fill this out!

In [ ]:
def train_realnvp(model, train_data, batch_size = 512, lr = 0.001, num_epochs = 5):
    dataset = torch.utils.data.TensorDataset(torch.Tensor(train_data).to(device))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_losses = []
    
    for i in range(num_epochs):
        for X_train in dataloader:
            optimizer.zero_grad()
            predictions, objective = model(X_train[0])
            
            # Define your total loss value here
            loss = # YOUR CODE HERE
            loss.backward()
            optimizer.step()
            
            train_losses.append(
                # Store the value of total NLL here
                # YOUR CODE HERE
            )
            
    print("Train NLL(bits)")
    plt.plot(train_losses, color='green')
    plt.show()

Create the model and launch training

In [ ]:
realnvp = RealNVP().to(device)

In [ ]:
train_realnvp(realnvp, X_train)

Now evaluate your model: calculate NLL in bits on your validation set.

In [ ]:
realnvp.eval()

# Implement NLL calculation on X_val data
################
# YOUR CODE HERE
###############
val_nll_bits = # YOUR CODE HERE
print("NLL on validation set in bits: {}".format(val_nll_bits))

Feel free to tune your model parameters to improve your NLL score!

Now it's time for some visualizations! Let's calculate flow values of our validation data.

In [ ]:
sampled_flow = realnvp(torch.Tensor(X_val).to(device))[0].detach().cpu().numpy()

Now we will visualize our points before and after passing throw the flow, leaving the label color.

In [ ]:
plt.scatter(X_val.T[0], X_val.T[1], c = Y_val)
plt.show()

In [ ]:
plt.scatter(sampled_flow.T[0], sampled_flow.T[1], c = Y_val)
plt.show()

Finally, let's perform sampling from RealNVP and visualize their histogram to compare with testing data.

In [ ]:
def sample_from_realnvp(model):
    x1 = random.random()
    x2 = random.random()
    latent = torch.Tensor([[x1, x2]]).to(device)
    result = realnvp.reverse(latent)[0]
    
    return result.detach().cpu().numpy()

In [ ]:
sampled_data = [sample_from_realnvp(realnvp) for _ in range(X_test.shape[0])]

In [ ]:
sampled_data = np.array(sampled_data)

In [ ]:
plt.hist2d(sampled_data.T[0], sampled_data.T[1], bins=100, cmap='gray',range=[[-3, 3], [-2, 3]])
plt.show()

In [ ]:
plt.hist2d(X_test.T[0], X_test.T[1], bins=100, cmap='gray')
plt.show()

# Future steps

This seminar doesn't cover high-dimensional case: working with distribution on images. However, it's not so different from 2D case. You are free to implement and try to run RealNVP or PixelCNN models on real images datasets: MNIST, CIFAR, CelebA.

To further deepen your knoledge in likelihood-based models we suggest you to take a look at task 3 in <a href="https://drive.google.com/open?id=1DtYllaV4Yk8ljgYcLBmdXNplEDTG6HT6">HW1</a> and task 2 in <a href="https://drive.google.com/file/d/1xs9fFCrPs3c9HNnOlmgen1ZnLfs26VVM/view?usp=sharing">HW2</a> of original Berkeley course and try to complete them by yourself.